In [ ]:
%%configure -f
{"name": "remotesparkmagics-sue", "executorMemory": "12G", "numExecutors": 10, 
 "executorCores": 4,
 "conf": {"spark.jars": "/system/jar/simpleHTM.jar,/system/jar/jhealpix.jar"}}

In [ ]:
import simple.HTMindex
import healpix.jhu.Healpix
import org.apache.spark.sql.{SparkSession, SaveMode, Row, DataFrame}
import org.apache.spark.sql.functions.udf
import java.util.Calendar

In [ ]:
object HTMUtils extends Serializable {
    var htmindex= new HTMindex() with Serializable
    val htmid: (Double, Double) => Long = htmindex.lookupId(_,_)

    val htmidUDF=udf(htmid)
    
}

object HEALPixUtils extends Serializable {
    var hp = new Healpix() with Serializable
    val healpixid: (Double, Double) => Long = hp.ang2pix(_,_)
    val healpixidUDF=udf(healpixid)
    
}

In [ ]:
print("hello from scala")

# can we get this login thing to work in scala?
this was a pain, it would be better if i ACTUALL knew scala i guess

In [ ]:


val authDF = spark.read.json("hdfs:///.config/creds.json")
lazy val m = authDF.first.getValuesMap[Any](authDF.schema.fieldNames)

lazy val username = (m("jdbc_username"))

lazy val password = (m("jdbc_password"))





# get data from parquet files into dataframe, add extra computed columns for htmid and healpixid

In [ ]:
val datafile = "/user/hive/warehouse/object"
val objDF = spark.read.parquet(datafile)

val newDF = objDF.withColumn("htmid",HTMUtils.htmidUDF(objDF("ra"),objDF("decl"))).withColumn("healpixid", HEALPixUtils.healpixidUDF(objDF("ra"), objDF("decl")))

newDF.write.format("parquet").mode("overwrite").saveAsTable("object_newcols")


